In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import time 

load_dotenv()

# Pinecone Configuration
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

spec = ServerlessSpec(cloud="aws", region="us-east-1")
index_name = "shop-product-catalog"

# Connect to the index
my_index=pc.Index(index_name)
time.sleep(1)

In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\DELL\AppData\Local\Temp\ipykernel_11376\638543296.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [21]:
from langchain_community.vectorstores import Pinecone

vectorstore = Pinecone(
    index=my_index,
    embedding=model,
    text_key="Description"    
)

In [22]:
query="What is the price of Gazelle Adidas product?"

result= vectorstore.similarity_search(query, k=1)
print(result)

[Document(metadata={'Gender': 'Women', 'Price': '10999', 'PrimaryColor': 'Purple', 'ProductBrand': 'Adidas', 'ProductName': 'Gazelle Pro 2'}, page_content='Retro-inspired running shoes with a sleek design and supportive upper')]


In [43]:
import google.generativeai as genai 
load_dotenv()

chat_history=[]

system_message=("""
    You are a helpful and respective shop assistant who answers queries relevant only to the products known to you.
    Please answer all the questions as a professional and customer-friendly tone. If a query lacks a direct answer related to the product,
    then generate a response based on related features. If a question is anything other than shopping, reply with, 'I can only provide answers related to the store only.' 
""")

In [33]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")

In [ ]:
def gen_answer(system_message, chat_history, prompt):
    """
    Generate AI response using Gemini model with memory of chat history.
    """

    # Build conversation messages in proper chat format
    chat = model.start_chat(history=[
            {"role": "user", "parts": [system_message]},
            *[
                {"role": "user" if msg.startswith("User:") else "model",
                 "parts": [msg.split(":", 1)[1].strip()]}
                for msg in chat_history
            ]
        ])
        
    # Generate response
    response = chat.send_message(prompt)
    answer = response.text.strip()

    # Update chat history
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"Assistant: {answer}")

    return answer

In [30]:
def get_relevant_chunk(query, vectorestore):
    results=vectorestore.similarity_search(query, k=1)
    if results:
        metadata=results[0].metadata
        context=(
            f'Product Name: {metadata.get("ProductName","Not Available")}\n'
            f'Brand: {metadata.get("Brand","Not Available")}\n'
            f'Price: {metadata.get("Price","Not Available")}\n'
            f'Color: {metadata.get("Color","Not Available")}\n'
            f'Description: {results[0].page_content}'
            )
        return context
    return "No relevant search"

def make_prompt(query,context):
    return f"Query: {query}\n\nContext:\n{context}\n\nAnswer:"

In [ ]:
def main():
    query="What is the price of Gazelle Adidas product?"
    relevant_text=get_relevant_chunk(query,vectorstore)
    prompt=make_prompt(query, relevant_text)

    answer=gen_answer(system_message,chat_history,prompt)
    print("Answer:",answer)

    
if __name__=="__main__":
    main()

Answer: Alright! We have the Gazelle Pro 2 available. The price is 10,999. Can I help you with anything else regarding these, like checking for available sizes or other details?
